# Thesis project notebook 

##### Uncomment and run the cell below to automatically install the required libraries

In [1]:
# %pip install -r requirements.txt

Start by importing the required libraries

In [2]:
import time
import pandas as pd
import utils
import json
import random

Let's load comments from a json file and see how many there are

In [3]:
from get_video import load_comments

comments = load_comments("comments_4DCbZJh-Gk4.json")
print(comments[:10])
print(f"There are {len(comments)} comments in this dataset")

Reading from local file <comments_4DCbZJh-Gk4.json>
['Outstanding!', 'my lord i imagine a war space i love this emg videos john browne are the best ever   ...ever suppppport  djent\nim from argentina say love metal instrumental', "1:31 that switch up is fucking timeless. I've known this song for about 5 years now and it still never gets old.", 'And yet we still search for proof of alien life 🤷🏻\u200d♂️', 'After almost 7 years this playthrough still brings tears to my eyes. So good and hope to see Monuments live again soon!', 'Djesus Christ', 'Masterpiece!!', 'god DAMN that part beginning at 1:58 is SOOO fucking good like holy FUCK man', 'All finger breaker chords. All the time.', 'Bruh']
There are 100 comments in this dataset


This might take some time depending on the size of our dataset, so let's define some checkpoints

In [4]:
checkpoints = {int(len(comments) * perc): perc * 100 for perc in [0.25, 0.5, 0.75, 0.9, 1]} # shamelessly stolen from Computational Linguistics Notebook 8

##### Now we can start analyzing them!

We will analyze the comments with BERT first, and save the results to a list. This is due to the fact that the BERT classifier pipeline works better when given a list of comments rather than a single comment

In [5]:
bert_scores = utils.bert_classifier(comments)

Now we analyze the comments iteratively using Vader and Textblob

In [6]:
start_time = time.time()

df_list = []

for i, comment in enumerate(comments):
        current_time = time.time()
        elapsed_time = current_time - start_time

        lang = utils.detect_language(comment)

        # Vader
        vader_score = utils.vader_classifier(comment)

        # TextBlob - Polarity is a a float between -1 and 1 where -1 is negative and 1 is positive
        blob_score = utils.textblob_classifier(comment)

        df_dict = {
            "Comment": comment,
            "Language": lang,
            "Vader score": vader_score,
            "Textblob score": blob_score,
        }

        df_list.append(df_dict)

        if i + 1 in checkpoints:
            print(f"{checkpoints[i+1] :.0f}% of the comments have been analyzed\nTime elapsed: {elapsed_time :.2f} seconds")  


25% of the comments have been analyzed
Time elapsed: 20.53 seconds
50% of the comments have been analyzed
Time elapsed: 36.17 seconds
75% of the comments have been analyzed
Time elapsed: 54.68 seconds
90% of the comments have been analyzed
Time elapsed: 64.21 seconds
100% of the comments have been analyzed
Time elapsed: 70.14 seconds


We should merge the results of both analyses into a single dataframe

In [7]:
df = pd.DataFrame(df_list)
series_bert = pd.Series(bert_scores)

df['BERT score'] = series_bert

Let's inspect the dataframe. Start by looking at the first 10 rows

In [8]:
df.head(10)

,Comment,Language,Vader score,Textblob score,BERT score
0,Outstanding!,de,0.6476,0.625000,0.896599
1,my lord i imagine a war space i love this emg ...,en,0.8658,0.666667,0.894711
2,1:31 that switch up is fucking timeless. I've ...,en,0.0000,-0.250000,-0.551378
3,And yet we still search for proof of alien lif...,en,0.0000,-0.250000,0.367565
4,After almost 7 years this playthrough still br...,en,0.7428,0.435227,0.736152
5,Djesus Christ,et,0.0000,0.000000,0.272853
6,Masterpiece!!,ro,0.6892,0.000000,0.879835
7,god DAMN that part beginning at 1:58 is SOOO f...,en,-0.1518,0.150000,-0.496737
8,All finger breaker chords. All the time.,en,0.0000,0.000000,0.759309
9,Bruh,id,0.0000,0.000000,-0.386926


And now let's check the stats for our numerical columns 

In [9]:
print(df.describe())

       Vader score  Textblob score  BERT score
count   100.000000      100.000000  100.000000
mean      0.245066        0.177426    0.246427
std       0.460005        0.421838    0.564940
min      -0.726200       -1.000000   -0.967942
25%       0.000000        0.000000   -0.289669
50%       0.080000        0.129018    0.385416
75%       0.682900        0.482924    0.723057
max       0.957100        1.000000    0.951091


The models seem to be predicting very similarly, but we should still check mathematically using Pearson correlation

In [10]:
print(df.corr(method='pearson'))

                Vader score  Textblob score  BERT score
Vader score        1.000000        0.667741    0.465176
Textblob score     0.667741        1.000000    0.561260
BERT score         0.465176        0.561260    1.000000


Now we have an idea of what our analysis is going to look like. However, one video is not enough. Let's build a bigger dataset using the top 50 most searched terms on YouTube

In [11]:
terms = pd.read_csv('search.tsv', sep='\t') #obtained from https://ahrefs.com/blog/top-youtube-searches/
terms.head(10)

,#,Keyword,Search Volume
0,1,bts,"16,723,304"
1,2,pewdiepie,"16,495,659"
2,3,asmr,"14,655,088"
3,4,billie eilish,"13,801,247"
4,5,baby shark,"12,110,100"
5,6,old town road,"10,456,524"
6,7,music,"10,232,134"
7,8,badabun,"10,188,997"
8,9,blackpink,"9,580,131"
9,10,fortnite,"9,117,342"


We can use these search terms to get YouTube video links which we can then use to gather comments. To save some time (and some precious API quota) we'll use the top 50 searched terms

DO NOT RUN THIS CELL or it will eat your API quota for this month

In [12]:
# from get_video import get_videos
# half = terms.head(50)
# search_terms = half["Keyword"]
# urls = []

# for term in search_terms:
#     urls.append(get_videos(term))

# links = list(set(item for sublist in urls for item in sublist))

# with open("videos__.json", "w") as f:
#     json.dump(links, f)

In [13]:
with open("videos__.json", "r") as f:
    links = json.load(f)

print(f"This file contains {len(links)} links")

This file contains 911 links


This is a lot of links (and therefore a lot of data). However, taking into account the rate of analysis and the amount of comments per video, we realize that it's going to take a lot of time: many of these videos have around 20,000 comments minimum. Knowing that it takes this program around a minute and a half to analyze 100 comments, analyzing all comments would take days. 

Instead of waiting for 9 days, we can make a compromise: we can use 500 comments and 500 comments per video. 

Let's start by randomly selecting 500 URLs.

In [14]:
random.seed(19)

half_links = random.sample(links, k=500)
links = list(half_links)

with open("links.json", "w") as f:
    links = json.dump(links, f)

Now that we have our list of links, we can start collecting data. Let's do the fast part first: collecting the like/dislike ratio and saving it somewhere convenient for later. 

As before, this cell should ideally be run once only as it takes a lot of time. If the notebook needs to be restarted, you should load the csv file instead.

In [15]:
# from get_video import get_likes

# start_time = time.time()

# with open("links.json", "r") as f:
#     links = json.load(f)
# df_list = []

# checkpoints = {int(len(links) * perc): perc * 100 for perc in [0.25, 0.5, 0.75, 0.9, 1]} # same as before

# for i, link in enumerate(links):
#     current_time = time.time()
#     elapsed_time = current_time - start_time
#     data = get_likes(link)
#     df_list.append(data)
#     time.sleep(0.5) # otherwise we will get an HTTP error
#     if i + 1 in checkpoints:
#             print(f"{checkpoints[i+1] :.0f}% of the videos have been analyzed\nTime elapsed: {elapsed_time :.2f} seconds")  
     
# df = pd.DataFrame(df_list)
# utils.save_analysis(df, "top50_ratio")

25% of the videos have been analyzed
Time elapsed: 72.85 seconds
50% of the videos have been analyzed
Time elapsed: 147.14 seconds
75% of the videos have been analyzed
Time elapsed: 222.05 seconds
90% of the videos have been analyzed
Time elapsed: 266.44 seconds
100% of the videos have been analyzed
Time elapsed: 296.23 seconds
saved to <analysis_top50_ratio.csv>


Let's look at the first 10 entries

In [17]:
df.head(10)

,URL,likes,dislikes
0,https://www.youtube.com/watch?v=ttWQK5VXskA,784677,25441
1,https://www.youtube.com/watch?v=5ula1NjaHUA,37445,723
2,https://www.youtube.com/watch?v=CPK_IdHe1Yg,3781735,145753
3,https://www.youtube.com/watch?v=QVLvD5_6ld0,224538,16484
4,https://www.youtube.com/watch?v=4bMOTTJqGgM,25754,9165
5,https://www.youtube.com/watch?v=KCwK2ECPDLA,42762,1603
6,https://www.youtube.com/watch?v=L40EFeYQmv8,85259,387
7,https://www.youtube.com/watch?v=Gl6ekgobG2k,444028,59264
8,https://www.youtube.com/watch?v=bheqvt6Z8S8,866,28
9,https://www.youtube.com/watch?v=evy4n3gW7LU,1015,22


This is good data, but it is not complete. We are still missing the ratio of likes to dislikes. Let's compute that now!

In [20]:
likes, dislikes = df['likes'], df['dislikes']

ratios = []

for like, dislike in zip(likes, dislikes):
    if like == 0 or dislike == 0:
        ratio = 0
        ratios.append(ratio)
    else:
        ratio = round(like / dislike, 3)
        ratios.append(ratio)

ratios = pd.Series(ratios)
ratios.head(10)

0     30.843
1     51.791
2     25.946
3     13.622
4      2.810
5     26.676
6    220.307
7      7.492
8     30.929
9     46.136
dtype: float64

Let's add this list to our dataframe

In [21]:
df['ratio'] = ratios
df.head(10)

,URL,likes,dislikes,ratio
0,https://www.youtube.com/watch?v=ttWQK5VXskA,784677,25441,30.843
1,https://www.youtube.com/watch?v=5ula1NjaHUA,37445,723,51.791
2,https://www.youtube.com/watch?v=CPK_IdHe1Yg,3781735,145753,25.946
3,https://www.youtube.com/watch?v=QVLvD5_6ld0,224538,16484,13.622
4,https://www.youtube.com/watch?v=4bMOTTJqGgM,25754,9165,2.810
5,https://www.youtube.com/watch?v=KCwK2ECPDLA,42762,1603,26.676
6,https://www.youtube.com/watch?v=L40EFeYQmv8,85259,387,220.307
7,https://www.youtube.com/watch?v=Gl6ekgobG2k,444028,59264,7.492
8,https://www.youtube.com/watch?v=bheqvt6Z8S8,866,28,30.929
9,https://www.youtube.com/watch?v=evy4n3gW7LU,1015,22,46.136
